In [ ]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
train = pd.read_csv("digit-recognizer/train.csv")

In [ ]:
print(train)

train_x = train.loc[:,"pixel0":]
train_y = train.loc[:,"label"]


In [ ]:
# show picture
one_pic = train.loc[2,"pixel0":]
one_pic = one_pic.to_numpy()
one_pic = one_pic.reshape(28,28)
print(one_pic.shape)

plt.matshow(one_pic, cmap=plt.get_cmap('gray'))

In [ ]:
train_x_reshape = []

for i in range(len(train_x)):
    train_x_reshape.append(train_x.loc[i,:].to_numpy().reshape(28,28,1))  # for cNN，need more dim 
train_x_reshape = np.array(train_x_reshape)

In [ ]:
# flatten version 

model = tf.keras.models.Sequential([

  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  
  tf.keras.layers.Dense(10),
])

In [ ]:
# cNN version
model = tf.keras.models.Sequential([

  tf.keras.layers.Conv2D(28, (3, 3), activation='relu', input_shape=(28, 28,1)),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10),
])

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_x_reshape, train_y, epochs=10)

In [ ]:
test = pd.read_csv("digit-recognizer/test.csv")

In [ ]:
test_x = test.loc[:,"pixel0":]


In [ ]:
test_x_reshape = []

for i in range(len(test_x)):
    test_x_reshape.append(test_x.loc[i,:].to_numpy().reshape(28,28,1))
test_x_reshape = np.array(test_x_reshape)

In [ ]:
import numpy as np
predict = model.predict(test_x_reshape)
test_y_hat = []
for i in range(len(predict)):
    test_y_hat.append(np.argmax(predict[i]))

In [ ]:
print(test_y_hat)

In [ ]:
predict = model.predict(train_x_reshape)
train_y_hat = []
for i in range(len(predict)):
    train_y_hat.append(np.argmax(predict[i]))

In [ ]:
m = tf.keras.metrics.Accuracy()
m.update_state(train_y_hat, train_y)
m.result().numpy()

# train 0.95730954 (128兩層)
# test  0.94128

# train 0.96859527 (512 五層)
# test  0.95603

# train 0.99609524 (CNN 3層)
# test 0.98367     
# 比較參數 512 五層 卻有 1,195,018
# CNN 3層  只有            90,978  個 trainable parameter 

# 這就是架構不同的力量 !! (CN有參考到2D相鄰資訊)

In [ ]:
# show the wrong case
import time
for i in range(len(train_x)):
    if train_y_hat[i] != train_y[i]:
        print(i)
        one_pic = train_x.loc[i,:]
        one_pic = one_pic.to_numpy()
        one_pic = one_pic.reshape(28,28)
        #print(one_pic.shape)
        print(train_y_hat[i] , train_y[i])
        plt.matshow(one_pic, cmap=plt.get_cmap('gray'))
        time.sleep(2)

In [ ]:
# try add one more KNN layer 

# make knn X and y 
train_x_ten_dim = model.predict(train_x_reshape)
test_x_ten_dim = model.predict(test_x_reshape)

# check point :  print(train_y_ten_dim[0] , train_y[0])



# KNN model
from sklearn.neighbors import KNeighborsClassifier


neigh = KNeighborsClassifier(n_neighbors=10)



neigh.fit(train_x_ten_dim, train_y)

# KNN效果提升  0.98367 ==> 0.98517
# 但其實沒有很明顯

In [ ]:
knn_train_y_hat = neigh.predict(train_x_ten_dim)



In [ ]:
m = tf.keras.metrics.Accuracy()
m.update_state(knn_train_y_hat, train_y)
m.result().numpy()  
# 提升0.99259526 ==> 0.9957857

In [ ]:
# show knn the wrong case
import time
for i in range(len(train_x)):
    if train_y_hat[i] != train_y[i]:
        print(i)
        one_pic = train_x.loc[i,:]
        one_pic = one_pic.to_numpy()
        one_pic = one_pic.reshape(28,28)
        #print(one_pic.shape)
        print(train_y_hat[i] , train_y[i])
        plt.matshow(one_pic, cmap=plt.get_cmap('gray'))
        time.sleep(0.5)

In [ ]:
test_y_hat = neigh.predict(test_x_ten_dim)

In [ ]:
# make answer.csv
temp = {"ImageId":[],"Label":[]}
count = 1

for i in test_y_hat:
    
    temp["ImageId"].append(count)
    
    #temp["SalePrice"].append((i)**2)
    temp["Label"].append(i)
    count+=1

temp = pd.DataFrame(temp)
!del answer.csv
temp.to_csv("answer.csv",index=False)